# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex='male',
       age=16,
       respiratory_rate=18,
       oxygen_saturation=99,
       heart_rate=72,
       systolic_bp=120,
       temperature=98.6,
       alertness='alert',
       inspired_oxygen='air',
       performance_status=	'Unrestricted normal activity'):
  sex=sex.lower()
  alertness=alertness.lower()
  inspired_oxygen=inspired_oxygen.lower()
  performance_status=performance_status.lower()
  score=0
  if sex=='male':
    score+=1
  elif sex=='female':
    score+0
  if age>=16 and age<50:
    score+=0
  elif age>=50 and age<66:
    score+=2
  elif age>=66 and age<81:
    score+=3
  elif age>=80:
    score+=4
  if respiratory_rate<9:
    score+=3
  elif respiratory_rate>=9 and respiratory_rate<12:
    score+=1
  elif respiratory_rate>=12 and respiratory_rate<21:
    score+=0
  elif respiratory_rate>=21 and respiratory_rate<25:
    score+=2
  elif respiratory_rate>=24:
    score+=3
  if oxygen_saturation >95:
    score+=0
  elif oxygen_saturation==94 or oxygen_saturation==95:
    score+=1
  elif oxygen_saturation==92 or oxygen_saturation==93:
    score+=2
  elif oxygen_saturation<92:
    score+=3
  if heart_rate<41:
    score+=3
  elif heart_rate>=41 and heart_rate<=50:
    score+=1
  elif heart_rate>=51 and heart_rate<=90:
    score+=0
  elif heart_rate>=91 and heart_rate<=110:
    score+=1
  elif heart_rate>=111 and heart_rate<=130:
    score+=2
  elif heart_rate>130:
    score+=3
  if systolic_bp<91:
    score+=3
  elif systolic_bp>=91 and systolic_bp<=100:
    score+=2
  elif systolic_bp>=101 and systolic_bp<=110:
    score+=1
  elif systolic_bp>=111 and systolic_bp<=219:
    score+=0
  elif systolic_bp>219:
    score+=3
  if temperature <35.1:
    score+=3
  elif temperature>=35.1 and temperature<=36.0:
    score+=1
  elif temperature>=36.1 and temperature<=38.0:
    score+=0
  elif temperature>=38.1 and temperature<=39.0:
    score+=1
  elif temperature>39.0:
    score+=2
  if alertness=='alert':
    score+=0
  elif alertness=='confused or not alert':
    score+=3
  if inspired_oxygen=='air':
    score+=0
  elif inspired_oxygen=='supplemental oxygen':
    score+=2
  if performance_status=='unrestricted normal activity':
    score+=0
  elif performance_status=='limited strenuous activity, can do light activity':
    score+=1
  elif performance_status=='limited activity, can self-care':
    score+=2
  elif performance_status=='limited self-care':
    score+=3
  elif performance_status=='bed/chair bound, no self-care':
    score+=4
  if score==0 or score==1:
    percentage=1
  elif score==2 or score==3:
    percentage=2
  elif score==4:
    percentage=3
  elif score==5:
    percentage=9
  elif score==6:
    percentage=15
  elif score==7:
    percentage=18
  elif score==8:
    percentage=22
  elif score==9:
    percentage=26
  elif score==10:
    percentage=29
  elif score==11:
    percentage=34
  elif score==12:
    percentage=38
  elif score==13:
    percentage=46
  elif score==14:
    percentage=47
  elif score==15:
    percentage=49
  elif score==16:
    percentage=55
  elif score>=17 and score<=25:
    percentage='0.59'
  elif score>=26:
    percentage='0.99'
  return percentage

In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [5]:
def find_hospital(age,sex,risk):

  sex=sex.lower()

  import requests
  url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={0}&sex={1}&risk_pct={2}".format(age,sex,risk)
  response = requests.get(url)
  return response.json()['hospital']


In [7]:
find_hospital(50,'male',0.54)

'Emory Dunwoody Medical Center'

In [ ]:
    import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [11]:
import requests
import json
def get_address(hospital='hospital'):
  url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
  response = requests.get(url)
  hospital_data=json.loads(response.text)
  for hosp in hospital_data.keys():
    if hosp==hospital:
      return hospital_data[hosp]
get_address("Emory Dunwoody Medical Center")

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---